# Tutorial: PromptFill Zero-to-Hero Content Masterclass

Audience:
- Builders, creators, and teams producing branded content with PromptFill + Remotion.

Prerequisites:
- Basic command-line comfort.
- Python 3.10+ for notebook execution.
- Repo cloned locally with dependencies installed.

What you will be able to do:
1. Orchestrate the full skill stack from setup to final renders.
2. Turn brand signals into structured scene systems.
3. Validate copy constraints automatically before rendering.
4. Run a quality loop that consistently ships higher-quality output.


## Outline

1. Dependency setup and initialization.
2. Environment and repo preflight.
3. Skill mission control (what to use and when).
4. Live demos from this repo:
   - Brand signal extraction analysis.
   - Copy and scene grammar validation.
   - Scene production board generation.
   - Asset gallery and export plan.
5. Interactive examples:
   - Scene timing playground.
   - Remotion skill demonstration lab.
6. Knowledge checkpoints with self-grading.
7. QA scorecard and iteration loop.
8. Exercises, capstone prompt generator, and next steps.

Target runtime: 60-120 minutes with labs.


In [ ]:
from __future__ import annotations

import json
import shutil
import subprocess
from datetime import datetime
from pathlib import Path
from typing import Any


def find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / '.git').exists():
            return candidate
    return start


REPO_ROOT = find_repo_root(Path.cwd()).resolve()
print(f"Repo root: {REPO_ROOT}")
print(f"Run timestamp: {datetime.now().isoformat(timespec='seconds')}")


def rel(path: Path) -> str:
    try:
        return str(path.resolve().relative_to(REPO_ROOT))
    except ValueError:
        return str(path)


def run(cmd: list[str], cwd: Path | None = None, check: bool = False) -> subprocess.CompletedProcess:
    return subprocess.run(
        cmd,
        cwd=str(cwd or REPO_ROOT),
        text=True,
        capture_output=True,
        check=check,
    )


## Step 0 - Dependency setup and initialization

This cell verifies and initializes notebook dependencies for a reproducible run.

It covers:
1. Python modules for notebook interactivity.
2. System tools (`node`, `npm`, `ffmpeg`, `gh`).
3. Node project dependencies for `web`, `video`, and `chatgpt-app`.
4. Jupyter Notebook skill environment variables.

By default this cell is **safe/read-only**. Flip install toggles only when you need bootstrap behavior.


In [ ]:
import os
import sys
import importlib.util

# Toggle these only when bootstrapping a fresh environment.
INSTALL_OPTIONAL_PYTHON_DEPS = False
INSTALL_NODE_DEPS = False

OPTIONAL_PYTHON_MODULES = ['ipywidgets', 'IPython']
SYSTEM_BINARIES = ['node', 'npm', 'python3', 'ffmpeg', 'gh']
NODE_PROJECTS = ['web', 'video', 'chatgpt-app']

# Skill environment setup (as recommended by jupyter-notebook skill)
os.environ.setdefault('CODEX_HOME', str(Path.home() / '.codex'))
os.environ.setdefault(
    'JUPYTER_NOTEBOOK_CLI',
    str(Path(os.environ['CODEX_HOME']) / 'skills/jupyter-notebook/scripts/new_notebook.py'),
)

print('Environment:')
print('- CODEX_HOME =', os.environ['CODEX_HOME'])
print('- JUPYTER_NOTEBOOK_CLI =', os.environ['JUPYTER_NOTEBOOK_CLI'])
print()


def has_module(name: str) -> bool:
    return importlib.util.find_spec(name) is not None


print('Python optional modules:')
missing_py = []
for mod in OPTIONAL_PYTHON_MODULES:
    ok = has_module(mod)
    print(f"- [{'OK' if ok else 'MISSING'}] {mod}")
    if not ok:
        missing_py.append(mod)

if missing_py and INSTALL_OPTIONAL_PYTHON_DEPS:
    print()
    print('Installing missing Python modules...')
    cmd = [sys.executable, '-m', 'pip', 'install', *missing_py]
    res = run(cmd)
    print('pip exit:', res.returncode)
    if res.stdout:
        print(chr(10).join(res.stdout.splitlines()[:10]))
    if res.stderr:
        print(chr(10).join(res.stderr.splitlines()[:10]))
elif missing_py:
    print()
    print('Optional modules missing. Set INSTALL_OPTIONAL_PYTHON_DEPS=True to install.')

print()
print('System binaries:')
for b in SYSTEM_BINARIES:
    found = shutil.which(b)
    print(f"- [{'OK' if found else 'MISSING'}] {b}: {found or 'not found'}")

print()
print('Node project dependencies:')
for project in NODE_PROJECTS:
    project_path = REPO_ROOT / project
    pkg = project_path / 'package.json'
    node_modules = project_path / 'node_modules'
    print(f"- {project}: package.json={'YES' if pkg.exists() else 'NO'}, node_modules={'YES' if node_modules.exists() else 'NO'}")

if INSTALL_NODE_DEPS:
    print()
    print('Installing Node dependencies for all projects...')
    for project in NODE_PROJECTS:
        cmd = ['npm', '--prefix', str(REPO_ROOT / project), 'install']
        res = run(cmd)
        print(f"- npm install {project}: exit={res.returncode}")
        if res.returncode != 0 and res.stderr:
            print('  stderr preview:', res.stderr.splitlines()[:5])
else:
    print()
    print('Node install skipped. Set INSTALL_NODE_DEPS=True to run npm installs.')


## Step 1 - Environment and repo preflight

We start with a deterministic preflight check. This prevents hidden setup drift from wasting time later.


In [ ]:
required_paths = [
    REPO_ROOT / 'docs/MASTER_TUTORIAL_ZERO_TO_HERO.md',
    REPO_ROOT / 'video/brand/stripe/brand-signals.json',
    REPO_ROOT / 'packages/brand-kit-stripe/src/copy/script.example.json',
    REPO_ROOT / 'video/public/ui/promptfill-ui-1280x720.png',
    REPO_ROOT / 'docs/gifs/promptfill-ui.gif',
]

print("Preflight file checks:")
for p in required_paths:
    status = "OK" if p.exists() else "MISSING"
    print(f"- [{status}] {rel(p)}")

print()
print("Binary/tool checks:")
for binary in ["node", "npm", "python3", "ffmpeg", "gh"]:
    found = shutil.which(binary)
    status = "OK" if found else "OPTIONAL/MISSING"
    print(f"- [{status}] {binary}: {found or 'not found'}")


## Step 2 - Skill mission control

This cell inspects where each required skill is installed and confirms readiness.


In [ ]:
skills = [
    "remotion-skill-brand-builder",
    "remotion-motion-director",
    "remotion-best-practices",
    "ralph-wiggum",
    "motion-designer",
    "video-motion-graphics",
    "create-remotion-geist",
    "remotion-ads",
    "elevenlabs-remotion",
    "remotion-resemble-ai",
    "lottie",
    "motion-canvas",
    "motion",
    "ffmpeg",
    "video-report",
    "writing-clearly-and-concisely",
]

skill_roots = [
    Path.home() / '.codex/skills',
    Path.home() / '.agents/skills',
    REPO_ROOT / '.agents/skills',
    REPO_ROOT / 'skills',
]

rows = []
for skill in skills:
    found_at = None
    for root in skill_roots:
        candidate = root / skill
        if candidate.exists():
            found_at = candidate
            break
    rows.append((skill, bool(found_at), rel(found_at) if found_at else 'not found'))

max_skill = max(len(r[0]) for r in rows)
print(f"{'Skill'.ljust(max_skill)}  Installed  Location")
print("-" * (max_skill + 34))
for skill, ok, location in rows:
    status = 'YES' if ok else 'NO'
    print(f"{skill.ljust(max_skill)}  {status.ljust(9)}  {location}")

missing = [s for s, ok, _ in rows if not ok]
print()
print("Missing skills:", missing if missing else 'None')


## Step 3 - Parse the master tutorial structure

We extract headings and phases from the Markdown playbook so we can build automation around the same structure.


In [ ]:
tutorial_path = REPO_ROOT / 'docs/MASTER_TUTORIAL_ZERO_TO_HERO.md'
text = tutorial_path.read_text(encoding='utf-8')
lines = text.splitlines()

h2 = [line for line in lines if line.startswith('## ')]
phases = [line for line in h2 if line.startswith('## Phase ')]

print(f"Tutorial file: {rel(tutorial_path)}")
print(f"Total lines: {len(lines)}")
print(f"H2 sections: {len(h2)}")
print(f"Phases: {len(phases)}")
print()
print("Phase order:")
for i, phase in enumerate(phases, start=1):
    print(f"{i}. {phase.replace('## ', '')}")


## Step 4 - Demo: brand signal extraction analysis

This demo reads real extracted brand signals and turns them into immediate direction cues.


In [ ]:
brand_signals_path = REPO_ROOT / 'video/brand/stripe/brand-signals.json'
brand = json.loads(brand_signals_path.read_text(encoding='utf-8'))
merged = brand.get('merged', {})

palette = merged.get('palette_candidates', [])[:8]
voice = merged.get('voice_keywords', [])[:8]
fonts = merged.get('font_candidates', [])

print(f"Brand slug: {brand.get('brand_slug')}")
print(f"Source URLs: {len(brand.get('inputs', []))}")
print(f"Palette candidates (top {len(palette)}):")
for c in palette:
    print(f"- {c['value']} (count={c['count']})")

print()
print("Voice keyword candidates:")
for kw in voice:
    print(f"- {kw['value']} (count={kw['count']})")

print()
print("Font candidates:", fonts if fonts else 'No strong signal captured from source pages')

try:
    from IPython.display import HTML, display

    swatches = ''.join(
        f"<div style='display:inline-block;margin:6px;text-align:center;'>"
        f"<div style='width:72px;height:36px;border:1px solid #aaa;border-radius:6px;background:{c['value']};'></div>"
        f"<div style='font-size:11px;margin-top:4px'>{c['value']}</div>"
        f"</div>"
        for c in palette
    )
    display(HTML(f"<div><strong>Palette preview</strong><div>{swatches}</div></div>"))
except Exception:
    pass


## Step 5 - Demo: validate script grammar and copy constraints

Professional output comes from strict copy constraints before visual design.


In [ ]:
script_path = REPO_ROOT / 'packages/brand-kit-stripe/src/copy/script.example.json'
script = json.loads(script_path.read_text(encoding='utf-8'))


def word_count(text: str) -> int:
    return len([w for w in text.strip().split() if w])


def validate_script(data: dict) -> list[str]:
    issues: list[str] = []
    scenes = data.get('scenes', [])

    required_ids = ['hook', 'problem', 'solution', 'cta']
    seen_ids = [scene.get('id') for scene in scenes]
    for rid in required_ids:
        if rid not in seen_ids:
            issues.append(f"Missing scene id: {rid}")

    for scene in scenes:
        sid = scene.get('id')
        if sid == 'hook':
            hook = scene.get('hook', '')
            wc = word_count(hook)
            if not (1 <= wc <= 8):
                issues.append(f"hook word count {wc} is outside 1-8")

        if sid == 'problem':
            bullets = scene.get('bullets', [])
            if not (1 <= len(bullets) <= 3):
                issues.append(f"problem bullet count {len(bullets)} is outside 1-3")
            for idx, bullet in enumerate(bullets, start=1):
                wc = word_count(str(bullet))
                if not (2 <= wc <= 4):
                    issues.append(f"problem bullet {idx} has {wc} words (needs 2-4)")

        if sid == 'solution':
            claim = scene.get('claim', '')
            proof = scene.get('proof', '')
            total_wc = word_count(claim) + word_count(proof)
            if not (1 <= total_wc <= 26):
                issues.append(f"solution total words {total_wc} exceed 26")

        if sid == 'cta':
            cta = scene.get('cta', '')
            wc = word_count(cta)
            if not (2 <= wc <= 4):
                issues.append(f"cta word count {wc} is outside 2-4")

    return issues


issues = validate_script(script)
print(f"Script file: {rel(script_path)}")
print(f"Scenes: {[s.get('id') for s in script.get('scenes', [])]}")
print()
print("Validation result:")
if issues:
    for issue in issues:
        print(f"- FAIL: {issue}")
else:
    print("- PASS: all enforced constraints satisfied")


## Step 6 - Demo: generate a production board from script

This converts script structure into an execution plan your editors and agents can follow.


In [ ]:
def build_board(data: dict) -> list[dict]:
    defaults = {
        'hook': ('HookScene', 90, 'fade+scale', 'remotion-motion-director'),
        'problem': ('ProblemSolution', 120, 'stagger+slide', 'motion-designer'),
        'solution': ('FeatureList', 120, 'spring+reveal', 'video-motion-graphics'),
        'cta': ('CTAEndCard', 90, 'hold+fade', 'remotion-ads'),
    }

    board = []
    for scene in data.get('scenes', []):
        sid = scene.get('id', 'unknown')
        template, duration, transition, owner_skill = defaults.get(
            sid,
            ('CustomScene', 90, 'fade', 'remotion-best-practices'),
        )
        board.append(
            {
                'scene_id': sid,
                'template': template,
                'duration_frames': duration,
                'transition': transition,
                'quality_owner': owner_skill,
            }
        )
    return board


board = build_board(script)
print('Scene production board:')
for idx, row in enumerate(board, start=1):
    print(
        f"{idx}. {row['scene_id']:>8} | template={row['template']:<16} | "
        f"frames={row['duration_frames']:<3} | transition={row['transition']:<14} | "
        f"owner={row['quality_owner']}"
    )

print()
print(f"Total duration (frames): {sum(r['duration_frames'] for r in board)}")
print(f"Approx duration @30fps: {sum(r['duration_frames'] for r in board)/30:.1f}s")


## Step 6.5 - Interactive scene timing playground (Remotion)

This lab lets you tune scene duration by hand and immediately see total runtime impact.

Use this for:
1. Balancing Hook/Problem/Solution/CTA pacing.
2. Comparing 9:16 short-form timing against longer explainer timing.
3. Testing whether your storyboard fits your target length before rendering.


In [ ]:
# Interactive timing lab
base_board = build_board(script)
base_durations = {row['scene_id']: row['duration_frames'] for row in base_board}


def duration_summary(durations: dict[str, int], fps: int = 30) -> dict[str, float]:
    total_frames = sum(int(v) for v in durations.values())
    total_seconds = total_frames / fps
    return {
        'total_frames': total_frames,
        'total_seconds': round(total_seconds, 2),
        'target_15s_fit': total_seconds <= 15,
        'target_30s_fit': total_seconds <= 30,
        'target_60s_fit': total_seconds <= 60,
    }


print('Default scene durations (frames):', base_durations)
print('Default summary @30fps:', duration_summary(base_durations, fps=30))

try:
    import ipywidgets as widgets
    from IPython.display import display

    sliders = {
        scene_id: widgets.IntSlider(
            value=frames,
            min=30,
            max=240,
            step=5,
            description=scene_id,
            continuous_update=False,
            style={'description_width': 'initial'},
        )
        for scene_id, frames in base_durations.items()
    }
    fps_widget = widgets.Dropdown(options=[24, 30, 60], value=30, description='fps')
    output = widgets.Output()

    def _refresh(*_):
        with output:
            output.clear_output()
            current = {k: int(v.value) for k, v in sliders.items()}
            summary = duration_summary(current, fps=int(fps_widget.value))
            print('Current durations:', current)
            print('Summary:', summary)
            if not summary['target_30s_fit']:
                print('Hint: tighten problem/solution beats for short-form ads.')

    for widget in [*sliders.values(), fps_widget]:
        widget.observe(_refresh, names='value')

    display(widgets.VBox([fps_widget, *sliders.values(), output]))
    _refresh()
except Exception:
    print('ipywidgets not available. Use this manual override pattern instead:')
    demo = {'hook': 75, 'problem': 105, 'solution': 105, 'cta': 75}
    print('Demo durations:', demo)
    print('Demo summary @30fps:', duration_summary(demo, fps=30))


## Step 6.6 - Remotion skill demonstration lab

This lab shows how each core Remotion skill contributes to production quality and where to verify outcomes.

It demonstrates:
1. `remotion-skill-brand-builder` for brand-system artifacts.
2. `remotion-motion-director` for pacing and direction controls.
3. `remotion-best-practices` for technical correctness.
4. `motion-designer` and `video-motion-graphics` for craft upgrades.
5. `ralph-wiggum` for iterative verification loops.


In [ ]:
skill_demo_catalog: dict[str, dict[str, Any]] = {
    'remotion-skill-brand-builder': {
        'goal': 'Generate reusable brand kit artifacts and script schema guardrails.',
        'artifact_checks': [
            'video/brand/stripe/brand-signals.json',
            'packages/brand-kit-stripe/src/brand/tokens.json',
            'packages/brand-kit-stripe/src/copy/schema.ts',
        ],
        'commands': [
            ['npm', 'run', 'brand:scan', '--', '--url', 'https://stripe.com', '--brand-slug', 'stripe'],
        ],
    },
    'remotion-motion-director': {
        'goal': 'Drive storyboard quality, pacing intent, and visual restraint.',
        'artifact_checks': [
            'skills/remotion-motion-director/SKILL.md',
            'docs/plans/2026-02-06-remotion-motion-director-skill-spec.md',
        ],
        'commands': [],
    },
    'remotion-best-practices': {
        'goal': 'Enforce clean Remotion implementation patterns and render correctness.',
        'artifact_checks': [
            'video/src/Root.tsx',
            'video/package.json',
        ],
        'commands': [
            ['npm', '--prefix', 'video', 'run', 'lint'],
        ],
    },
    'motion-designer': {
        'goal': 'Apply animation principles to avoid robotic motion.',
        'artifact_checks': [
            '.agents/skills/motion-designer/SKILL.md',
        ],
        'commands': [],
    },
    'video-motion-graphics': {
        'goal': 'Apply motion graphics rhythm, staging, and reveal systems.',
        'artifact_checks': [
            '.agents/skills/video-motion-graphics/SKILL.md',
        ],
        'commands': [],
    },
    'ralph-wiggum': {
        'goal': 'Run repeated build/verify loops until criteria pass.',
        'artifact_checks': [
            '/Users/danielgreen/.codex/skills/ralph-wiggum/SKILL.md',
        ],
        'commands': [
            ['npm', '--prefix', 'web', 'run', 'test'],
        ],
    },
}


def demo_remotion_skill(skill_name: str, execute_commands: bool = False) -> None:
    entry = skill_demo_catalog.get(skill_name)
    if not entry:
        print(f'Unknown skill: {skill_name}')
        print('Available:', sorted(skill_demo_catalog.keys()))
        return

    print(f"Skill: {skill_name}")
    print(f"Goal: {entry['goal']}")
    print('Artifact checks:')
    for raw in entry['artifact_checks']:
        path = Path(raw)
        if not path.is_absolute():
            path = REPO_ROOT / raw
        status = 'OK' if path.exists() else 'MISSING'
        print(f"- [{status}] {rel(path)}")

    cmds = entry.get('commands', [])
    if not cmds:
        print('No command demo configured for this skill.')
        return

    print('Command demos:')
    for cmd in cmds:
        print('  $', ' '.join(cmd))
        if execute_commands:
            result = run(cmd)
            print('    exit:', result.returncode)
            if result.stdout:
                print('    stdout preview:', result.stdout.strip().splitlines()[:3])
            if result.stderr:
                print('    stderr preview:', result.stderr.strip().splitlines()[:3])


# Non-destructive default demonstration
demo_remotion_skill('remotion-skill-brand-builder', execute_commands=False)
print()
demo_remotion_skill('remotion-best-practices', execute_commands=False)

try:
    import ipywidgets as widgets
    from IPython.display import display

    skill_dropdown = widgets.Dropdown(
        options=sorted(skill_demo_catalog.keys()),
        value='remotion-skill-brand-builder',
        description='Skill',
        style={'description_width': 'initial'},
    )
    execute_toggle = widgets.Checkbox(value=False, description='Execute demo commands')
    run_button = widgets.Button(description='Run demo', button_style='primary')
    output = widgets.Output()

    def _on_click(_):
        with output:
            output.clear_output()
            demo_remotion_skill(skill_dropdown.value, execute_commands=execute_toggle.value)

    run_button.on_click(_on_click)
    display(widgets.VBox([skill_dropdown, execute_toggle, run_button, output]))
except Exception:
    print()
    print('ipywidgets not available. Call demo_remotion_skill(<name>, execute_commands=False) manually.')


## Step 6.7 - Demonstration: `remotion-best-practices` capabilities

This module demonstrates what `remotion-best-practices` enforces in practice:

1. Frame-driven animation (`useCurrentFrame`, no CSS animation shortcuts).
2. Composition hygiene in `Root.tsx`.
3. Timing and interpolation patterns aligned with Remotion primitives.
4. Rule coverage map for what to load next when solving specific problems.


In [ ]:
best_skill_path = Path('/Users/danielgreen/.codex/skills/remotion-best-practices/SKILL.md')
rules_root = Path('/Users/danielgreen/.codex/skills/remotion-best-practices/rules')

focus_rules = [
    'animations.md',
    'compositions.md',
    'timing.md',
    'transitions.md',
    'gifs.md',
]

print('Skill path:', best_skill_path)
print('Exists:', best_skill_path.exists())
print()
print('Focused rule files:')
for name in focus_rules:
    rp = rules_root / name
    print(f"- [{'OK' if rp.exists() else 'MISSING'}] {rp}")

# Demonstration checks on this repo
print()
print('Repo capability checks:')
use_current = run(['rg', '-n', r'useCurrentFrame\(', 'video/src'])
composition_defs = run(['rg', '-n', '<Composition', 'video/src/Root.tsx'])
css_anim = run(['rg', '-n', '(animation:|@keyframes|animate-)', 'video/src'])

use_current_lines = [l for l in use_current.stdout.splitlines() if l.strip()]
composition_lines = [l for l in composition_defs.stdout.splitlines() if l.strip()]
css_anim_lines = [l for l in css_anim.stdout.splitlines() if l.strip()]

print(f"- useCurrentFrame usage count: {len(use_current_lines)}")
print(f"- Composition definitions in Root.tsx: {len(composition_lines)}")
print(f"- CSS animation-like patterns found in video/src: {len(css_anim_lines)}")

if use_current_lines:
    print('  sample useCurrentFrame refs:', use_current_lines[:3])
if composition_lines:
    print('  sample Composition refs:', composition_lines[:3])
if css_anim_lines:
    print('  sample CSS animation refs:', css_anim_lines[:3])
else:
    print('  no forbidden CSS animation patterns detected in video/src')


## Step 6.8 - Demonstration: `create-remotion-geist` capabilities

This module demonstrates Geist-specific rules and lint checks:

1. Dark-theme token conventions.
2. Spring-based motion defaults.
3. Critical constraints (no emoji, proper icons, `.tsx` entrypoint).
4. Quick “Geist compliance” lint on example snippets.


In [ ]:
geist_skill_path = Path('/Users/danielgreen/.agents/skills/create-remotion-geist/SKILL.md')
geist_setup_ref = Path('/Users/danielgreen/.agents/skills/create-remotion-geist/references/project-setup.md')
geist_scene_ref = Path('/Users/danielgreen/.agents/skills/create-remotion-geist/references/scene-patterns.md')

print('Skill path:', geist_skill_path)
print('Exists:', geist_skill_path.exists())
print('Reference setup exists:', geist_setup_ref.exists())
print('Reference scene patterns exists:', geist_scene_ref.exists())

skill_text = geist_skill_path.read_text(encoding='utf-8') if geist_skill_path.exists() else ''

critical_rules = []
in_critical = False
for line in skill_text.splitlines():
    if line.strip().startswith('## Critical Rules'):
        in_critical = True
        continue
    if in_critical and line.strip().startswith('## '):
        break
    if in_critical and line.strip().startswith(('1.', '2.', '3.', '4.')):
        critical_rules.append(line.strip())

print()
print('Critical rules extracted:')
for r in critical_rules:
    print('-', r)


def geist_lint(code_snippet: str, is_entrypoint: bool = False) -> list[str]:
    issues: list[str] = []

    # Emoji detection (basic range)
    if any(ord(ch) > 10000 for ch in code_snippet):
        issues.append('Contains emoji-like characters; use Geist icons instead.')

    if '@geist-ui/icons' not in code_snippet:
        issues.append('Missing @geist-ui/icons import for icon usage.')

    if is_entrypoint and 'registerRoot' not in code_snippet:
        issues.append('Entrypoint missing registerRoot() call.')

    if 'regex-based syntax highlighting' in code_snippet.lower():
        issues.append('Mentions regex-based syntax highlighting (forbidden by skill).')

    return issues


good_scene = """
import { Check } from '@geist-ui/icons';
import { AbsoluteFill, useCurrentFrame, useVideoConfig } from 'remotion';

export const Scene = () => {
  const frame = useCurrentFrame();
  const { fps } = useVideoConfig();
  return <AbsoluteFill style={{ backgroundColor: '#0a0a0a' }}><Check /></AbsoluteFill>;
};
"""

bad_scene = """
import { AbsoluteFill } from 'remotion';

export const Scene = () => {
  return <AbsoluteFill>🚀 fast launch</AbsoluteFill>;
};
"""

print()
print('Geist lint (good scene):', geist_lint(good_scene))
print('Geist lint (bad scene):', geist_lint(bad_scene))


## Step 6.9 - Demonstration: `remotion-motion-director` capabilities

This module demonstrates the required output contract from `remotion-motion-director`:

1. Creative direction brief.
2. Brand + motion control sheet.
3. Scene-by-scene storyboard.
4. Build notes mapped to Remotion primitives.
5. QA rubric score and improvement deltas.


In [ ]:
motion_director_path = Path('/Users/danielgreen/.codex/skills/remotion-motion-director/SKILL.md')
brand_schema_ref = Path('/Users/danielgreen/.codex/skills/remotion-motion-director/references/brand-control-schema.md')
rubric_ref = Path('/Users/danielgreen/.codex/skills/remotion-motion-director/references/production-rubric.md')

print('Motion director skill exists:', motion_director_path.exists())
print('Brand schema exists:', brand_schema_ref.exists())
print('Production rubric exists:', rubric_ref.exists())


def build_motion_director_artifacts(script_data: dict) -> dict[str, Any]:
    scenes = script_data.get('scenes', [])

    creative_brief = {
        'message': 'Move revenue faster with one brand system.',
        'audience': script_data.get('audience', 'busy operators'),
        'visual_promise': 'Clean, premium, high-clarity motion with restrained transitions.',
    }

    control_sheet = {
        'motion_profile_primary': 'measured-premium',
        'motion_profile_fallback': 'calm-minimal',
        'pace': 'medium',
        'safe_margin_px': 80,
        'max_parallel_animated_elements': 3,
        'reduced_motion_variant': True,
    }

    storyboard = []
    cursor = 0
    for s in scenes:
        sid = s.get('id', 'unknown')
        duration = 90 if sid in {'hook', 'cta'} else 120
        storyboard.append(
            {
                'scene_id': sid,
                'start_frame': cursor,
                'end_frame': cursor + duration,
                'intent': f"Deliver {sid} scene with one clear job.",
            }
        )
        cursor += duration

    build_notes = [
        'Use Composition in Root.tsx for each format target.',
        'Use Sequence for scene timing boundaries.',
        'Use spring/interpolate for all reveal motion.',
        'Apply one shared motion token set per project.',
    ]

    rubric_scores = {
        'Narrative Clarity': 2,
        'Brand Fidelity': 2,
        'Motion Craft': 1,
        'Pacing and Rhythm': 1,
        'Accessibility and Readability': 2,
        'Platform Safety': 2,
        'Audio/VO Integration': 1,
        'Emotional Arc': 1,
        'Technical Stability': 2,
        'Distinctiveness': 1,
    }
    total = sum(rubric_scores.values())

    deltas = [
        'Increase contrast separation in secondary text blocks.',
        'Refine transition cadence between problem and solution scenes.',
        'Add one stronger emotional beat before CTA hold.',
    ]

    return {
        'creative_brief': creative_brief,
        'control_sheet': control_sheet,
        'storyboard': storyboard,
        'build_notes': build_notes,
        'qa': {
            'scores': rubric_scores,
            'total': total,
            'threshold': 16,
            'status': 'pass' if total >= 16 else 'needs-improvement',
            'next_pass_deltas': deltas,
        },
    }


artifacts = build_motion_director_artifacts(script)
print('Creative brief:', artifacts['creative_brief'])
print()
print('Control sheet:', artifacts['control_sheet'])
print()
print('Storyboard first two scenes:', artifacts['storyboard'][:2])
print()
print('QA total:', artifacts['qa']['total'], 'status:', artifacts['qa']['status'])
print('Next-pass deltas:', artifacts['qa']['next_pass_deltas'])


## Step 6.10 - Anatomy: `remotion-skill-brand-builder`

This anatomy explorer breaks the skill into operational parts:

1. Trigger conditions and default strategy.
2. Workflow phases (intake, scan, kit, starter, validation).
3. Guardrails.
4. Output standard and repo contract checks.


In [ ]:
brand_builder_skill = REPO_ROOT / '.agents/skills/remotion-skill-brand-builder/SKILL.md'
question_bank = REPO_ROOT / '.agents/skills/remotion-skill-brand-builder/references/question-bank.md'
output_contract = REPO_ROOT / '.agents/skills/remotion-skill-brand-builder/references/output-contract.md'

print('Skill docs present:')
for pth in [brand_builder_skill, question_bank, output_contract]:
    print(f"- [{'OK' if pth.exists() else 'MISSING'}] {rel(pth)}")

text = brand_builder_skill.read_text(encoding='utf-8') if brand_builder_skill.exists() else ''
lines = text.splitlines()

workflow_steps = [ln.strip() for ln in lines if ln.strip().startswith('### ')]
guardrails = []
output_standard = []
in_guardrails = False
in_output = False
for ln in lines:
    s = ln.strip()
    if s.startswith('## Guardrails'):
        in_guardrails = True
        in_output = False
        continue
    if s.startswith('## Output Standard'):
        in_guardrails = False
        in_output = True
        continue
    if s.startswith('## ') and not s.startswith('## Guardrails') and not s.startswith('## Output Standard'):
        in_guardrails = False
        in_output = False

    if in_guardrails and s.startswith('- '):
        guardrails.append(s[2:])
    if in_output and s and s[0].isdigit() and s[1:2] == '.':
        output_standard.append(s)

print()
print('Workflow anatomy:')
for step in workflow_steps:
    print('-', step.replace('### ', ''))

print()
print('Guardrails:')
for g in guardrails:
    print('-', g)

print()
print('Output standard:')
for o in output_standard:
    print('-', o)

required_core_paths = [
    REPO_ROOT / 'packages/brand-kit-stripe/package.json',
    REPO_ROOT / 'packages/brand-kit-stripe/src/index.ts',
    REPO_ROOT / 'packages/brand-kit-stripe/src/brand/tokens.json',
    REPO_ROOT / 'packages/brand-kit-stripe/src/copy/schema.ts',
    REPO_ROOT / 'packages/brand-kit-stripe/src/starter/ScriptDrivenReel.tsx',
    REPO_ROOT / 'tools/create-remotion-brand-kit-starter.mjs',
]

print()
print('Output contract checks (stripe):')
for pth in required_core_paths:
    print(f"- [{'OK' if pth.exists() else 'MISSING'}] {rel(pth)}")

try:
    import ipywidgets as widgets
    from IPython.display import display

    steps = [w.replace('### ', '') for w in workflow_steps]
    dropdown = widgets.Dropdown(options=steps, description='Workflow step', style={'description_width': 'initial'})
    out = widgets.Output()

    def _show(*_):
        with out:
            out.clear_output()
            selected = dropdown.value
            print('Selected step:', selected)
            if selected.startswith('1)'):
                print('Focus: structured interview from question-bank fast/full tracks.')
            elif selected.startswith('2)'):
                print('Focus: brand signal scan + manual verification of extracted hints.')
            elif selected.startswith('3)'):
                print('Focus: build portable @brand-kit package and enforce schema constraints.')
            elif selected.startswith('4)'):
                print('Focus: starter scaffold generation (embed/dependency modes).')
            elif selected.startswith('5)'):
                print('Focus: lint/build/render checks and consistency verification.')

    dropdown.observe(_show, names='value')
    display(widgets.VBox([dropdown, out]))
    _show()
except Exception:
    print()
    print('ipywidgets not available. Use printed anatomy and checks above.')


## Knowledge checkpoints (self-grading)

Use this checkpoint before moving to production renders.

You should pass at least 4/5 before shipping.


In [ ]:
knowledge_questions = [
    {
        'id': 'q1',
        'question': 'Which skill should lead brand-system generation?',
        'options': {'A': 'remotion-best-practices', 'B': 'remotion-skill-brand-builder', 'C': 'ffmpeg'},
        'answer': 'B',
    },
    {
        'id': 'q2',
        'question': 'In create-remotion-geist, which icon rule is correct?',
        'options': {'A': 'Use emojis for speed', 'B': 'Use @geist-ui/icons instead of emojis', 'C': 'Use any icon font'},
        'answer': 'B',
    },
    {
        'id': 'q3',
        'question': 'What is a core remotion-best-practices animation rule?',
        'options': {'A': 'Prefer CSS keyframes', 'B': 'Use useCurrentFrame() driven animation', 'C': 'Use Tailwind animate-* classes'},
        'answer': 'B',
    },
    {
        'id': 'q4',
        'question': 'What does remotion-motion-director require before coding?',
        'options': {'A': 'Storyboard with timing and intent', 'B': 'Immediate rendering', 'C': 'Only color token selection'},
        'answer': 'A',
    },
    {
        'id': 'q5',
        'question': 'Which remotion-motion-director artifact is mandatory?',
        'options': {'A': 'QA rubric score + next-pass deltas', 'B': 'No QA output needed', 'C': 'Only final MP4'},
        'answer': 'A',
    },
    {
        'id': 'q6',
        'question': 'Brand-builder output contract includes which package shape?',
        'options': {'A': '@brand-kit/<brand> with schema and starter reel', 'B': 'Single markdown file', 'C': 'Only video renders'},
        'answer': 'A',
    },
    {
        'id': 'q7',
        'question': 'Which behavior matches brand-builder guardrails?',
        'options': {'A': 'Copy competitor visuals exactly', 'B': 'Treat scraping as hints, then verify manually', 'C': 'Skip verification'},
        'answer': 'B',
    },
    {
        'id': 'q8',
        'question': 'What is the healthiest default scene design constraint?',
        'options': {'A': 'One scene, one primary purpose', 'B': 'Stack all messages in every scene', 'C': 'Unlimited bullets'},
        'answer': 'A',
    },
]


def grade_checkpoint(answers: dict[str, str]) -> tuple[int, int, list[str]]:
    total = len(knowledge_questions)
    score = 0
    feedback = []
    for q in knowledge_questions:
        user_answer = answers.get(q['id'], '').upper().strip()
        correct = q['answer']
        if user_answer == correct:
            score += 1
            feedback.append(f"{q['id']}: correct")
        else:
            feedback.append(f"{q['id']}: expected {correct}, got {user_answer or 'blank'}")
    return score, total, feedback


# Default attempt (edit this dict for manual grading)
attempt = {'q1': 'B', 'q2': 'B', 'q3': 'B', 'q4': 'A', 'q5': 'A', 'q6': 'A', 'q7': 'B', 'q8': 'A'}
score, total, feedback = grade_checkpoint(attempt)
print(f"Score: {score}/{total}")
for line in feedback:
    print('-', line)

try:
    import ipywidgets as widgets
    from IPython.display import display

    radio_widgets = {}
    for q in knowledge_questions:
        opts = [f"{k}: {v}" for k, v in q['options'].items()]
        radio_widgets[q['id']] = widgets.RadioButtons(
            options=opts,
            description=q['id'],
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='95%'),
        )

    submit = widgets.Button(description='Grade checkpoint', button_style='success')
    out = widgets.Output()

    def _grade(_):
        answers = {}
        for q in knowledge_questions:
            val = radio_widgets[q['id']].value
            answers[q['id']] = val.split(':', 1)[0].strip() if val else ''
        s, t, fb = grade_checkpoint(answers)
        with out:
            out.clear_output()
            print(f'Score: {s}/{t}')
            if s >= int(0.75 * t):
                print('Checkpoint passed. Proceed to production render loop.')
            else:
                print('Checkpoint not passed yet. Review weak areas and retry.')
            for line in fb:
                print('-', line)

    submit.on_click(_grade)
    items = []
    for q in knowledge_questions:
        items.append(widgets.HTML(value=f"<b>{q['id']}</b> {q['question']}"))
        items.append(radio_widgets[q['id']])
    display(widgets.VBox(items + [submit, out]))
except Exception:
    print('ipywidgets not available. Edit the attempt dict and rerun this cell.')


## Step 7 - Demo: visual asset gallery from real project captures

These are existing assets in this repo that you can use for previews, docs, and review loops.


In [ ]:
asset_paths = [
    REPO_ROOT / 'video/public/ui/promptfill-ui-1280x720.png',
    REPO_ROOT / 'video/public/ui/promptfill-drawer-1280x720.png',
    REPO_ROOT / 'video/public/ui/promptfill-share-1280x720.png',
    REPO_ROOT / 'docs/gifs/promptfill-ui.gif',
    REPO_ROOT / 'docs/gifs/promptfill-drawer.gif',
    REPO_ROOT / 'docs/gifs/promptfill-share.gif',
]

print('Asset inventory:')
for p in asset_paths:
    status = 'OK' if p.exists() else 'MISSING'
    size = p.stat().st_size if p.exists() else 0
    print(f"- [{status}] {rel(p)} ({size:,} bytes)")

try:
    from IPython.display import Image, display

    for p in asset_paths:
        if p.exists() and p.suffix.lower() in {'.png', '.gif', '.jpg', '.jpeg'}:
            print()
            print(f"Preview: {rel(p)}")
            display(Image(filename=str(p)))
except Exception:
    print()
    print("IPython display unavailable in this runtime. Paths above are valid for notebook rendering.")


## Step 8 - Demo: command deck generator

This creates a practical runbook you can paste into terminal sessions.


In [ ]:
command_deck = [
    {
        'phase': 'Setup',
        'commands': [
            'npm --prefix web install',
            'npm --prefix video install',
            'npm --prefix chatgpt-app install',
        ],
    },
    {
        'phase': 'Validation',
        'commands': [
            'npm --prefix web run lint',
            'npm --prefix web run test',
            'npm --prefix video run lint',
            'npm --prefix video run build',
            'npm --prefix chatgpt-app test',
        ],
    },
    {
        'phase': 'Brand builder',
        'commands': [
            'npm run brand:scan -- --url https://stripe.com --brand-slug stripe',
            'npm run brand:starter -- --out ./starter-stripe --mode embed',
        ],
    },
    {
        'phase': 'Render',
        'commands': [
            'cd video && npm run render:stripe:reels',
            'cd video && npm run render:stripe:yt',
        ],
    },
    {
        'phase': 'Preview packaging',
        'commands': [
            'ffmpeg -y -i renders/your-master.mp4 -vf "fps=12,scale=960:-1:flags=lanczos" docs/media/your-preview.gif',
        ],
    },
]

for section in command_deck:
    print()
    print(f"[{section['phase']}]")
    for cmd in section['commands']:
        print(f"  $ {cmd}")

missing_bins = [b for b in ['npm', 'node', 'ffmpeg'] if shutil.which(b) is None]
print()
print("Binary readiness:", 'All required binaries found' if not missing_bins else f'Missing: {missing_bins}')


## Step 9 - Demo: QA scorecard workbook

Use this rubric every iteration. It gives the `ralph-wiggum` loop an objective finish line.


In [ ]:
rubric = {
    'message_clarity_first_2s': 0,
    'scene_focus_one_job_each': 0,
    'hierarchy_consistency': 0,
    'motion_quality_and_rhythm': 0,
    'copy_conciseness': 0,
    'cta_strength': 0,
    'audio_mix_quality': 0,
    'export_quality': 0,
}

# Example scoring for a first pass (1-5 scale)
rubric.update(
    {
        'message_clarity_first_2s': 4,
        'scene_focus_one_job_each': 4,
        'hierarchy_consistency': 3,
        'motion_quality_and_rhythm': 3,
        'copy_conciseness': 5,
        'cta_strength': 4,
        'audio_mix_quality': 0,
        'export_quality': 4,
    }
)

max_score = len(rubric) * 5
total = sum(rubric.values())
pct = (total / max_score) * 100

print('QA Scorecard (1-5 each, 0 if not applicable yet):')
for k, v in rubric.items():
    print(f"- {k:32} : {v}")

print()
print(f"Total: {total}/{max_score} ({pct:.1f}%)")
print('Status:', 'READY FOR SHIP' if pct >= 80 else 'ITERATE AGAIN')


## Exercises

1. Replace the Stripe script with your own `script.json` and rerun validation.
2. Add one new scene type and extend `build_board()` for it.
3. Improve one weak rubric category by making a concrete implementation change.
4. Re-score and compare before/after percentages.


In [ ]:
# Exercise scaffold
# 1) Point this to your own script.
my_script_path = REPO_ROOT / 'packages/brand-kit-stripe/src/copy/script.example.json'
my_script = json.loads(my_script_path.read_text(encoding='utf-8'))

# 2) Run validations.
my_issues = validate_script(my_script)
print('Issues:', my_issues if my_issues else 'None')

# 3) Build board and inspect total timing.
my_board = build_board(my_script)
print('Scene count:', len(my_board))
print('Total frames:', sum(s['duration_frames'] for s in my_board))


## Pitfalls and extensions

Common pitfalls:

1. Skipping script constraints and trying to fix overflow in design.
2. Mixing too many transition styles in one video.
3. Changing typography per scene and losing brand cohesion.
4. Treating brand scan output as final truth rather than directional signal.

Extensions:

1. Add a second brand and compare two boards side by side.
2. Add voice generation and auto-duration adjustment.
3. Add a final packaging cell that exports multiple aspect ratios.


In [ ]:
def build_capstone_prompt(
    content_type: str,
    audience: str,
    output_format: str,
    energy: str,
    voice: str,
) -> str:
    sep = chr(10)
    return sep.join(
        [
            f"We are creating {content_type} for {audience} in {output_format} format.",
            f"Motion energy: {energy}. Brand voice: {voice}.",
            "",
            "Use skills in this order:",
            "1) $remotion-skill-brand-builder",
            "2) $remotion-motion-director",
            "3) $remotion-best-practices",
            "4) $motion-designer",
            "5) $video-motion-graphics",
            "6) $ralph-wiggum",
            "",
            "Deliver:",
            "- Brand control sheet and storyboard.",
            "- Script JSON validated against constraints.",
            "- Updated compositions and render commands.",
            "- QA scorecard with one improvement pass.",
        ]
    )


print(
    build_capstone_prompt(
        content_type='a 30-second product demo',
        audience='busy operators evaluating workflow tools',
        output_format='Reels 9:16 + YouTube 16:9',
        energy='medium',
        voice='confident, concise, practical',
    )
)


## Wrap-up

You now have a complete production notebook that bridges strategy, copy, design systems, motion craft, and QA.

Recommended next move:

1. Duplicate this notebook per campaign.
2. Keep one canonical QA rubric threshold for your team.
3. Archive your best script + board combinations as reusable templates.
